# Initial Setup

In [66]:
import sys
sys.path.insert(0, "../..")

In [67]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
PATH="/diskA/jethro/nips-papers"

In [105]:
import pandas as pd
import numpy as np
import tensorflow as tf
from urop.prodlda import ProdLDA
from gensim.corpora import Dictionary
from gensim.corpora.mmcorpus import MmCorpus
from gensim.utils import chunkize
from utils.array import chunks

In [70]:
dct = Dictionary.load(f'{PATH}/dictionary.pkl')

In [71]:
corpus = MmCorpus(f'{PATH}/corpus.mm')

In [72]:
articles = []
for article in corpus:
    arr = np.zeros(len(dct))
    for idx, count in article:
        arr[idx] = count
    articles.append(arr)
articles = np.array(articles)

In [73]:
len(articles)

7241

In [85]:
NUM_TOPICS = 50
LAYER_1_SIZE = 100
LAYER_2_SIZE = 100
BATCH_SIZE = 200
LEARNING_RATE = 2e-4

In [86]:
NETWORK_ARCHITECTURE = \
    dict(n_hidden_recog_1=LAYER_1_SIZE,
         n_hidden_recog_2=LAYER_2_SIZE,
         n_hidden_gener_1=articles.shape[1],
         n_input=articles.shape[1],
         n_z=NUM_TOPICS)

In [87]:
def create_minibatch(data, batch_size):
    rng = np.random.RandomState(10)

    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
    yield data[ixs]

In [123]:
def train(network_architecture, minibatches, training_epochs=100, display_step=5):
    tf.reset_default_graph()
    vae = ProdLDA(network_architecture, learning_rate=LEARNING_RATE, batch_size=BATCH_SIZE)
    emb=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        for batch in minibatches:
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch)
            # Compute average loss
#             avg_cost += cost / n_samples_tr * batch_size

#             if np.isnan(avg_cost):
#                 print(epoch,i,np.sum(batch_xs,1).astype(np.int),batch.shape)
#                 print('Encountered NaN, stopping training. Please check the learning_rate settings and the momentum.')
#                 # return vae,emb
#                 sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost))
    return vae, emb

In [124]:
def print_top_words(beta, feature_names, n_top_words=10):
    print('---------------Printing the Topics------------------')
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))
    print('---------------End of Topics------------------')

In [125]:
def calcPerp(model, docs_test):
    cost=[]
    for doc in docs_test:
        doc = doc.astype('float32')
        n_d = np.sum(doc)
        c=model.test(doc)
        cost.append(c/n_d)
    print('The approximated perplexity is: ',(np.exp(np.mean(np.array(cost)))))

In [126]:
minibatches = chunks(articles, BATCH_SIZE)

In [127]:
vae, emb = train(network_architecture, minibatches)

Learning Rate: 0.0002
Tensor("Softmax_1:0", shape=(200, 54254), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


InvalidArgumentError: Incompatible shapes: [41,54254] vs. [200,54254]
	 [[Node: gradients/mul_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](gradients/mul_grad/Shape, gradients/mul_grad/Shape_1)]]

Caused by op 'gradients/mul_grad/BroadcastGradientArgs', defined at:
  File "/home/jethro/anaconda3/envs/urop/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jethro/anaconda3/envs/urop/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jethro/anaconda3/envs/urop/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jethro/anaconda3/envs/urop/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jethro/anaconda3/envs/urop/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start

In [114]:
print_top_words(emb, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])

NameError: name 'emb' is not defined

In [ ]:
calcPerp(vae)